In [1]:
import os
if "models" not in os.listdir("."):
    os.chdir("../..")

In [2]:
%load_ext autoreload
%autoreload 2
import penzai
import jax_smi
jax_smi.initialise_tracking()
from penzai import pz
pz.ts.register_as_default()
pz.ts.register_autovisualize_magic()
pz.enable_interactive_context()

In [3]:
from micrlhf.llama import LlamaTransformer
llama = LlamaTransformer.from_pretrained("models/gemma-2b-it.gguf", from_type="gemma", load_eager=True, device_map="tpu:0")

In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("alpindale/gemma-2b")
tokenizer.padding_side = "right"

In [5]:
from sprint.task_vector_utils import load_tasks, ICLRunner
tasks = load_tasks()

In [6]:
def check_if_single_token(token):
    return len(tokenizer.tokenize(token)) == 1

task_name = "es_en"

task = tasks[task_name]

print(len(task))

task = {
    k:v for k,v in task.items() if check_if_single_token(k) and check_if_single_token(v)
}

print(len(task))

pairs = list(task.items())

763
346


In [7]:
def logprob_loss(logits, tokens, sep=1599, pad_token=32000, n_first=None, shift=None):
    logits = logits[:, :-1]

    # print(
    #     logits.argmax(axis=-1)
    # )

    logits = jnp.take_along_axis(logits, tokens[:, 1:, None], axis=-1).squeeze(-1)

    mask = tokens[:, 1:] == sep
    mask = jnp.cumsum(mask[:, ::-1], axis=-1)[:, ::-1] > 0
    mask = jnp.logical_not(mask)

    if shift is not None:
        rolled_mask = jnp.roll(mask, shift, axis=-1)
        mask = jnp.logical_and(mask, rolled_mask)

    # print(mask[:, -5:])
    
    if n_first is not None:
        rolled_mask = jnp.roll(mask, n_first, axis=-1)
        mask = jnp.logical_and(mask, jnp.logical_not(rolled_mask))

    mask = jnp.logical_and(mask, tokens[:, 1:] != pad_token)

    logits = logits * mask

    return logits.sum(axis=-1).mean(axis=-1)

In [8]:
from functools import partial

sep = 3978
pad = 0

def metric_fn(logits, resids, tokens):
    return logprob_loss(logits, tokens, sep=sep, pad_token=pad, n_first=2)

In [9]:
from micrlhf.llama import LlamaBlock, LlamaAttention
from micrlhf.utils.activation_manipulation import ActivationAddition, wrap_vector
from functools import partial
import jax.numpy as jnp
from penzai import pz
import jax

@partial(jax.jit, static_argnames=("metric", "batched"))
def run_with_add(additions_pre, additions_mid, tokens, metric, batched=False, llama=None):
    get_resids = llama.select().at_instances_of(LlamaBlock).apply_with_selected_index(lambda i, x:
        pz.nn.Sequential([
            pz.de.TellIntermediate.from_config(tag=f"resid_pre_{i}"),
            x
        ])
    )
    get_resids = get_resids.select().at_instances_of(LlamaBlock).apply_with_selected_index(lambda l, b: b.select().at_instances_of(pz.nn.Residual).apply_with_selected_index(lambda i, x: x if i == 0 else pz.nn.Sequential([
        pz.de.TellIntermediate.from_config(tag=f"resid_mid_{l}"),
        x,
    ])))


    get_resids = get_resids.select().at_instances_of(LlamaAttention).apply_with_selected_index(lambda i, x: x.select().at_instances_of(pz.nn.Softmax).apply(lambda b: pz.nn.Sequential([
        b,
        pz.de.TellIntermediate.from_config(tag=f"attn_{i}"),
    ])))

    get_resids = pz.de.CollectingSideOutputs.handling(get_resids, tag_predicate=lambda x: True)
    make_additions = get_resids.select().at_instances_of(LlamaBlock).apply_with_selected_index(lambda i, x:
        pz.nn.Sequential([
            ActivationAddition(pz.nx.wrap(additions_pre[i], *(("batch",) if batched else ()), "seq", "embedding"), "all"),
            x
        ])
    )
    make_additions = make_additions.select().at_instances_of(LlamaBlock).apply_with_selected_index(lambda l, b: b.select().at_instances_of(pz.nn.Residual).apply_with_selected_index(lambda i, x: x if i == 0 else pz.nn.Sequential([
        ActivationAddition(pz.nx.wrap(additions_mid[l], *(("batch",) if batched else ()), "seq", "embedding"), "all"),
        x,
    ])))
    tokens_wrapped = pz.nx.wrap(tokens, "batch", "seq")
    logits, resids = make_additions(llama.inputs.from_basic_segments(tokens_wrapped))
    return metric(logits.unwrap("batch", "seq", "vocabulary"), resids, tokens), (logits, resids[::3], resids[1::3], resids[2::3])


@partial(jax.jit, static_argnames=("metric",))
def get_metric_resid_grad(tokens, llama=llama, metric=metric_fn):
    additions = [jnp.zeros(tokens.shape + (llama.config.hidden_size,)) for _ in range(llama.config.num_layers)]
    batched = tokens.ndim > 1
    (metric, (logits, resids_pre, qk, resids_mid)), (grad_pre, grad_mid) = jax.value_and_grad(run_with_add, argnums=(0, 1), has_aux=True)(additions, additions, tokens, metric, batched=batched, llama=llama)
    return (
        metric,
        [r.value.unwrap("batch", "seq", "embedding") for r in resids_pre],
        [r.value.unwrap("batch", "seq", "embedding") for r in resids_mid],
        [r.value.unwrap("batch", "kv_heads", "q_rep", "seq", "kv_seq") for r in qk],
        grad_pre,
        grad_mid
    )


In [10]:
batch_size = 8 
n_shot=20
max_seq_len = 128
seed = 10

In [11]:
prompt = "Follow the pattern:\n{}"

In [12]:
runner = ICLRunner(task_name, pairs, batch_size=batch_size, n_shot=n_shot, max_seq_len=max_seq_len, seed=seed, prompt=prompt)

In [13]:
from sprint.task_vector_utils import tokenized_to_inputs

train_tokens = runner.get_tokens(
    runner.train_pairs, tokenizer
)["input_ids"]

In [14]:
metric_value, resids_pre, resids_mid, qk, grad_pre, grad_mid = get_metric_resid_grad(train_tokens, llama=llama)

In [15]:
get_rms_block = lambda layer, resid_index: (
    llama.select()
    .at_instances_of(LlamaBlock).pick_nth_selected(layer)
    .at_instances_of(pz.nn.Residual).pick_nth_selected(resid_index)
    .at_instances_of(pz.nn.RMSLayerNorm).pick_nth_selected(0)
    ).get()

In [16]:
mlp_rms = [get_rms_block(layer, 1) for layer in range(llama.config.num_layers)]

In [17]:
from micrlhf.utils.load_sae import get_nev_it_sae_suite

In [18]:
from micrlhf.utils.load_sae import weights_to_resid, resids_to_weights

In [19]:
from micrlhf.utils.load_sae import sae_encode_gated

In [20]:
def mlp_normalize(layer, resid_mid):
    # return resid_mid / resids_mid_norms[layer] * mlp_rms_weights[layer]
    # return resid_mid / jnp.linalg.norm(resid_mid, axis=-1, keepdims=True) * mlp_rms_weights[layer]
    return mlp_rms[layer](pz.nx.wrap(resid_mid, "batch", "seq", "embedding")).unwrap("batch", "seq", "embedding")

In [46]:
from tqdm.auto import tqdm

ie_attn = {}
sae_grads_attn = {}
ie_resid = {}
sae_grads_resid = {}
ie_transcoder = {}
sae_grads_transcoder = {}

ie_error_attn = {}
ie_error_resid = {}
ie_error_transcoder = {}

sae_error_attn = {}
sae_error_resid = {}
sae_error_transcoder = {}

def sfc_simple(grad, resid, target, sae):
    pre_relu, post_relu, recon = sae_encode_gated(sae, resid)
    error = target - recon
    f = partial(weights_to_resid, sae=sae)

    sae_grad, = jax.vjp(f, post_relu)[1](grad,)
    indirect_effects = sae_grad * post_relu
    indirect_effects_error = jnp.einsum("...f, ...f -> ...", grad, error)
    return indirect_effects, indirect_effects_error, sae_grad, error


layers = list(range(6, 17))
for layer in tqdm(layers):
    r_pre, r_mid, g_mid = resids_pre[layer], resids_mid[layer], grad_mid[layer]
    sae = get_nev_it_sae_suite(layer=layer, label="attn_out")
    indirect_effects, indirect_effects_error, sae_grad, error = sfc_simple(g_mid, r_mid - r_pre, r_mid - r_pre, sae)
    # display((indirect_effects > 0).sum(-1))
    ie_attn[layer] = indirect_effects
    ie_error_attn[layer] = indirect_effects_error
    sae_grads_attn[layer] = sae_grad
    sae_error_attn[layer] = error

# for layer, (r_pre, g_pre) in enumerate(zip(resids_pre, grad_pre)):
for layer in tqdm(layers):
    r_pre, g_pre = resids_pre[layer], grad_pre[layer]
    sae = get_nev_it_sae_suite(layer=layer)
    indirect_effects, indirect_effects_error, sae_grad, error = sfc_simple(g_pre, r_pre, r_pre, sae)
    # display((indirect_effects != 0).sum(-1))
    ie_resid[layer] = indirect_effects
    ie_error_resid[layer] = indirect_effects_error
    sae_grads_resid[layer] = sae_grad
    sae_error_resid[layer] = error

for layer in tqdm(layers[:-1]):
    r_mid, r_pre, g_pre = resids_mid[layer], resids_pre[layer + 1], grad_pre[layer + 1]
    sae = get_nev_it_sae_suite(layer=layer, label="transcoder")
    indirect_effects, indirect_effects_error, sae_grad, error = sfc_simple(g_pre, mlp_normalize(layer, r_mid), r_pre - r_mid, sae)
    # display((indirect_effects != 0).sum(-1))
    ie_transcoder[layer] = indirect_effects
    ie_error_transcoder[layer] = indirect_effects_error
    sae_grads_transcoder[layer] = sae_grad
    sae_error_transcoder[layer] = error

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [22]:
tokens_wrapped = pz.nx.wrap(train_tokens, "batch", "seq")
llama_inputs = llama.inputs.from_basic_segments(tokens_wrapped)

In [23]:
# def attn_call(layer, resid_pre):
#     subblock = llama.select().at_instances_of(LlamaBlock).pick_nth_selected(layer).at_instances_of(pz.nn.Residual).pick_nth_selected(0).get().delta
#     subblock = subblock.select().at_instances_of(LlamaAttention).get()

#     si_selection = subblock.select().at_instances_of(pz.de.HandledSideInputRef)
#     keys = sorted(set([ref.tag for ref in si_selection.get_sequence()]))
#     replaced = si_selection.apply(lambda ref: pz.de.SideInputRequest(tag=ref.tag))
#     subblock = pz.de.WithSideInputsFromInputTuple.handling(replaced, keys)

#     side_inputs = {
#         'positions': llama_inputs.positions,
#         'attn_mask': llama_inputs.attention_mask
#     }
    
#     resid_pre = resid_pre / 

#     resid_pre = pz.nx.wrap(resid_pre, "batch", "seq", "embedding")
#     attn_out = subblock((resid_pre,) + tuple(side_inputs[tag] for tag in subblock.side_input_tags))

#     attn_out = attn_out.unwrap("batch", "seq", "embedding") 

#     return attn_out.astype(resid.dtype)

# attn_call(6, resids_pre[6])

In [24]:
prompt_length = len(tokenizer.tokenize(prompt))
periods = ["input", "arrow", "output", "newline"]
masks = {
    "prompt": jnp.zeros_like(train_tokens).at[:, :prompt_length].set(1).astype(bool),
    **{
        period: jnp.zeros_like(train_tokens).at[:, prompt_length+i::len(periods)].set(1).astype(bool) * (train_tokens != pad) for i, period in enumerate(periods)
    }
}

In [25]:
def mask_average(vector, mask):
    mask = masks[mask]
    while mask.ndim < vector.ndim:
        mask = mask[..., None]

    return ((mask * vector).sum(1) / mask.sum(1)).mean(0)

In [26]:
def transcoder_feature_to_mid(layer, feature_idx, mask):
    sae = get_nev_it_sae_suite(layer=layer, label="transcoder")
    resid = resids_mid[layer]

    def f(resid):
        resid = mlp_normalize(layer, resid)
        batch_token_feat = resids_to_weights(resid, sae)[:, :, feature_idx] * sae_grads_transcoder[layer][:, :, feature_idx]
        token_act = mask_average(batch_token_feat, mask)
        return token_act

    return jax.grad(f)(resid)

In [27]:
def transcoder_error_to_mid(layer, mask):
    sae = get_nev_it_sae_suite(layer=layer, label="transcoder")
    resid_next = resids_pre[layer + 1]
    resid = resids_mid[layer]

    grad = grad_pre[layer + 1]

    def f(resid):
        _, _, recon = sae_encode_gated(sae, resid)
        err_by_grad = jnp.einsum("...f, ...f -> ...", (resid_next - recon), grad)
        return mask_average(err_by_grad, mask)

    return jax.grad(f)(resid)

In [28]:
def attn_out_feature_to_pre(layer, feature_idx, mask):
    sae = get_nev_it_sae_suite(layer=layer, label="attn_out")

    resid = resids_pre[layer]

    subblock = llama.select().at_instances_of(LlamaBlock).pick_nth_selected(layer).at_instances_of(pz.nn.Residual).pick_nth_selected(0).get().delta

    si_selection = subblock.select().at_instances_of(pz.de.HandledSideInputRef)
    keys = sorted(set([ref.tag for ref in si_selection.get_sequence()]))
    replaced = si_selection.apply(lambda ref: pz.de.SideInputRequest(tag=ref.tag))
    subblock = pz.de.WithSideInputsFromInputTuple.handling(replaced, keys)

    side_inputs = {
        'positions': llama_inputs.positions,
        'attn_mask': llama_inputs.attention_mask
    }

    def f(resid):
        resid = pz.nx.wrap(resid, "batch", "seq", "embedding")
        attn_out = subblock((resid,) + tuple(side_inputs[tag] for tag in subblock.side_input_tags))

        attn_out = attn_out.unwrap("batch", "seq", "embedding") 

        batch_token_feat = resids_to_weights(attn_out, sae)[:, :, feature_idx] * sae_grads_attn[layer][:, :, feature_idx]
        token_act = mask_average(batch_token_feat, mask)
        return token_act

    return jax.grad(f)(resid)

def attn_out_error_to_pre(layer, mask):
    sae = get_nev_it_sae_suite(layer=layer, label="attn_out")

    resid = resids_pre[layer]

    subblock = llama.select().at_instances_of(LlamaBlock).pick_nth_selected(layer).at_instances_of(pz.nn.Residual).pick_nth_selected(0).get().delta

    si_selection = subblock.select().at_instances_of(pz.de.HandledSideInputRef)
    keys = sorted(set([ref.tag for ref in si_selection.get_sequence()]))
    replaced = si_selection.apply(lambda ref: pz.de.SideInputRequest(tag=ref.tag))
    subblock = pz.de.WithSideInputsFromInputTuple.handling(replaced, keys)

    side_inputs = {
        'positions': llama_inputs.positions,
        'attn_mask': llama_inputs.attention_mask
    }

    def f(resid):
        resid = pz.nx.wrap(resid, "batch", "seq", "embedding")
        attn_out = subblock((resid,) + tuple(side_inputs[tag] for tag in subblock.side_input_tags))

        attn_out = attn_out.unwrap("batch", "seq", "embedding") 

        _, _, recon = sae_encode_gated(sae, attn_out)
        batch_token_feat = jnp.einsum("...f, ...f -> ...", attn_out - recon, grad_mid[layer])
        token_act = mask_average(batch_token_feat, mask)
        return token_act

    return jax.grad(f)(resid)
# float(jnp.linalg.norm(attn_out_error_to_pre(6, "arrow")))

In [29]:
def pre_feature_to_pre(layer, feature_idx, mask):
    sae = get_nev_it_sae_suite(layer=layer)
    resid = resids_pre[layer]

    def f(resid):
        batch_token_feat = resids_to_weights(resid, sae)[:, :, feature_idx] * sae_grads_resid[layer][:, :, feature_idx]
        token_act = mask_average(batch_token_feat, mask)
        return token_act

    return jax.grad(f)(resid)

def pre_error_to_pre(layer, mask):
    sae = get_nev_it_sae_suite(layer=layer)
    resid = resids_pre[layer]

    def f(resid):
        _, _, recon = sae_encode_gated(sae, resid)
        batch_token_error = jnp.einsum("...f, ...f -> ...", (resid - recon), grad_pre[layer])
        token_grad = mask_average(batch_token_error, mask)
        return token_grad

    return jax.grad(f)(resid)


In [34]:
def ie_pre_to_transcoder_features(layer, grad, mask):
    sae = get_nev_it_sae_suite(layer=layer, label="transcoder")
    resid_mid = resids_mid[layer]
    resid_mid = mlp_normalize(layer, resid_mid)
    ie = sfc_simple(grad, resid_mid, resid_mid, sae)[0]
    ie = mask_average(ie, mask)

    return ie

def ie_pre_to_transcoder_error(layer, grad, mask):
    sae = get_nev_it_sae_suite(layer=layer, label="transcoder")
    resid_next = resids_pre[layer + 1]
    resid_mid = resids_mid[layer]
    ie = sfc_simple(grad, mlp_normalize(layer, resid_mid), resid_next - resid_mid, sae)[1]
    ie = mask_average(ie, mask)

    return ie

In [38]:
def ie_mid_to_attn_features(layer, grad, mask):
    sae = get_nev_it_sae_suite(layer=layer, label="attn_out")
    resid_mid = resids_mid[layer]
    resid_pre = resids_pre[layer]

    ie = sfc_simple(grad, resid_mid - resid_pre, resid_mid - resid_pre, sae)[0]
    ie = mask_average(ie, mask)
    return ie

def ie_mid_to_attn_error(layer, grad, mask):
    sae = get_nev_it_sae_suite(layer=layer, label="attn_out")
    resid_mid = resids_mid[layer]
    resid_pre = resids_pre[layer]

    ie = sfc_simple(grad, resid_mid - resid_pre, resid_mid - resid_pre, sae)[1]
    ie = mask_average(ie, mask)
    return ie

In [55]:
def ie_pre_to_pre_features(layer, grad, mask):
    sae = get_nev_it_sae_suite(layer=layer)
    resid = resids_pre[layer]
    ie = sfc_simple(grad, resid, resid, sae)[0]
    ie = mask_average(ie, mask)
    return ie

def ie_pre_to_pre_error(layer, grad, mask):
    sae = get_nev_it_sae_suite(layer=layer)
    resid = resids_pre[layer]
    ie = sfc_simple(grad, resid, resid, sae)[1]
    ie = mask_average(ie, mask)
    return ie
# float((ie_pre_to_pre_features(6, grad_pre[6], "arrow") - mask_average(ie_error_resid[6], "arrow")).sum())

In [31]:
ie_pre_to_pre_features(6, grad_pre[6], "prompt")

# jax.Array float32(32768,) ≈-1.3e-08 ±2.6e-05 [≥-0.0023, ≤0.0023] zero:32_682 nonzero:86
  Array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [41]:
def grad_through_transcoder(layer, grad):
    sae = get_nev_it_sae_suite(layer, label="transcoder")
    resid_mid = resids_mid[layer]

    def f(resid_mid):
        resid_mid = mlp_normalize(layer, resid_mid)
        # we ignore error nodes
        weights = resids_to_weights(resid_mid, sae)
        recon = weights_to_resid(weights, sae)

        return recon

    grad = jax.vjp(f, resid_mid)[1](grad,)[0]

    return grad


In [38]:
def grad_through_mlp(layer, grad):
    mlp = llama.select().at_instances_of(LlamaBlock).pick_nth_selected(layer).at_instances_of(pz.nn.Residual).pick_nth_selected(1).get().delta
    resid_mid = resids_mid[layer]
    def f(resid_mid):
        resids_mid = pz.nx.wrap(resid_mid, "batch", "seq", "embedding")
        out = mlp(resids_mid)
        return out.unwrap("batch", "seq", "embedding").astype(resid_mid.dtype)
    return jax.vjp(f, resid_mid)[1](grad.astype(resid_mid.dtype),)[0]

In [42]:
def grad_through_attn(layer, grad):
    subblock = llama.select().at_instances_of(LlamaBlock).pick_nth_selected(layer).at_instances_of(pz.nn.Residual).pick_nth_selected(0).get().delta

    si_selection = subblock.select().at_instances_of(pz.de.HandledSideInputRef)
    keys = sorted(set([ref.tag for ref in si_selection.get_sequence()]))
    replaced = si_selection.apply(lambda ref: pz.de.SideInputRequest(tag=ref.tag))
    subblock = pz.de.WithSideInputsFromInputTuple.handling(replaced, keys)

    side_inputs = {
        'positions': llama_inputs.positions,
        'attn_mask': llama_inputs.attention_mask
    }

    def f(resid):
        resid_pre = pz.nx.wrap(resid, "batch", "seq", "embedding")
        attn_out = subblock((resid_pre,) + tuple(side_inputs[tag] for tag in subblock.side_input_tags))

        attn_out = attn_out.unwrap("batch", "seq", "embedding") 

        return attn_out.astype(resid.dtype)

    resid = resids_pre[layer]
    return jax.vjp(f, resid)[1](grad.astype(resid.dtype),)[0]

In [35]:
ies = ie_resid[12]
ies = mask_average(ies, "arrow")

In [36]:
jax.lax.top_k(ies, 30)

[# jax.Array float32(30,) ≈0.0012 ±0.0012 [≥0.0003, ≤0.0052] nonzero:30
   Array([0.00515363, 0.00363038, 0.00307088, 0.00272385, 0.00263665,
          0.00251921, 0.00248203, 0.00158354, 0.00141052, 0.00093801,
          0.00084221, 0.000733  , 0.00071481, 0.00070495, 0.0006948 ,
          0.00065009, 0.00064819, 0.00064437, 0.00058913, 0.00056076,
          0.00055267, 0.00053456, 0.00050256, 0.00044708, 0.00040716,
          0.00033888, 0.00031714, 0.00030809, 0.00030625, 0.00030374],      dtype=float32)
 ,
 # jax.Array int32(30,) [≥507, ≤32_734] nonzero:30
   Array([ 9157,  1112, 23723, 13869, 16983, 11386, 14154, 28260, 14214,
          11958, 19580, 10812, 23598,  3310, 19827,   507,  7525, 21048,
          16736, 20601, 24603, 14220,  6282, 23657, 23808, 22231, 21623,
          15223, 11506, 32734], dtype=int32)
 ]

In [56]:
from tqdm import trange


layer = 12
feature_idx = 9157
mask = "arrow"

resid_grad = pre_feature_to_pre(layer, feature_idx, mask)
feature_effects = {}
for layer in trange(layer - 1, 5, -1):
    for mask in masks:
        feature_effects[("t", layer, mask)] = ie_pre_to_transcoder_features(layer, resid_grad, mask)
        feature_effects[("et", layer, mask)] = ie_pre_to_transcoder_error(layer, resid_grad, mask)
    # # does not work # resid_grad = resid_grad - grad_through_mlp(layer, resid_grad)
    # resid_grad = resid_grad + grad_through_mlp(layer, resid_grad)
    for mask in masks:
        feature_effects[("a", layer, mask)] = ie_mid_to_attn_features(layer, resid_grad, mask)
        feature_effects[("ea", layer, mask)] = ie_mid_to_attn_error(layer, resid_grad, mask)
    # # does not work # resid_grad = resid_grad - grad_through_attn(layer, resid_grad)
    # resid_grad = resid_grad + grad_through_attn(layer, resid_grad)
    for mask in masks:
        feature_effects[("r", layer, mask)] = ie_pre_to_pre_features(layer, resid_grad, mask)
        feature_effects[("er", layer, mask)] = ie_pre_to_pre_error(layer, resid_grad, mask)

  0%|          | 0/6 [00:00<?, ?it/s]

100%|██████████| 6/6 [00:02<00:00,  2.19it/s]


In [59]:
from tqdm import tqdm
top_effects = []
k = 32
abs_effects = False
for key, features in tqdm(feature_effects.items()):
    if features.ndim == 0:
        top_effects.append((float(features), key, 0))
        continue
    effects, indices = jax.lax.top_k(features if not abs_effects else jnp.abs(features), k)
    for i, e in zip(indices.tolist(), effects.tolist()):
        top_effects.append((e, key, i))
top_effects.sort(reverse=True)

  0%|          | 0/180 [00:00<?, ?it/s]

100%|██████████| 180/180 [00:03<00:00, 53.87it/s]


In [60]:
top_effects[:k]

[(3.825262683676556e-05, ('er', 10, 'arrow'), 0),
 (3.696179192047566e-05, ('er', 9, 'arrow'), 0),
 (3.6785961128771305e-05, ('er', 11, 'arrow'), 0),
 (3.483711407170631e-05, ('er', 8, 'arrow'), 0),
 (3.0039775083423592e-05, ('er', 7, 'arrow'), 0),
 (2.6046691345982254e-05, ('er', 6, 'arrow'), 0),
 (1.864851583377458e-05, ('ea', 9, 'arrow'), 0),
 (1.303459248447325e-05, ('ea', 6, 'arrow'), 0),
 (1.0022412425314542e-05, ('t', 11, 'arrow'), 11868),
 (5.725710252590943e-06, ('r', 11, 'arrow'), 20013),
 (5.312870598572772e-06, ('ea', 7, 'arrow'), 0),
 (5.241968210611958e-06, ('r', 7, 'arrow'), 6814),
 (4.156298928137403e-06, ('et', 8, 'arrow'), 0),
 (3.868626208713977e-06, ('t', 11, 'arrow'), 7583),
 (3.6351477774587693e-06, ('r', 8, 'arrow'), 6063),
 (3.332952928758459e-06, ('r', 9, 'arrow'), 16780),
 (2.9112695756339235e-06, ('r', 9, 'arrow'), 19243),
 (2.852165607691859e-06, ('r', 7, 'arrow'), 16810),
 (2.563620455475757e-06, ('et', 10, 'arrow'), 0),
 (2.54974816016329e-06, ('r', 8, 'arrow'), 18678),
 (2.323580702068284e-06, ('r', 11, 'arrow'), 28800),
 (2.101049176417291e-06, ('r', 10, 'arrow'), 15973),
 (1.9500989765219856e-06, ('r', 7, 'arrow'), 9432),
 (1.9355418316990836e-06, ('r', 6, 'arrow'), 21082),
 (1.923181116580963e-06, ('r', 11, 'arrow'), 24640),
 (1.8755390556179918e-06, ('r', 10, 'arrow'), 8426),
 (1.8105280332747498e-06, ('r', 11, 'arrow'), 3184),
 (1.810401727198041e-06, ('r', 10, 'arrow'), 24460),
 (1.7926603277373943e-06, ('t', 10, 'arrow'), 17854),
 (1.775706095941132e-06, ('a', 11, 'arrow'), 30066),
 (1.7411440467185457e-06, ('r', 8, 'arrow'), 8442),
 (1.7342554201604798e-06, ('r', 6, 'arrow'), 2369)]